In [23]:
#cargamos las librerias que vamos a utilizar
import numpy as np
import gzip
import nltk
import json
import pandas as pd 
import ast
import matplotlib as plt
from textblob import TextBlob
import pyarrow
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [24]:
#cargamos el archivo json
with open('C:\\Users\\Dr-Pc\\OneDrive\\Escritorio\\PI MLOps - STEAM\\australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data = [eval(line) for line in file]

In [25]:
# Desanidar las columnas de 'items' en el DataFrame
user_reviews = pd.DataFrame(data)
user_reviews_explored = user_reviews.explode('reviews')
user_reviews_normalized = pd.json_normalize(user_reviews_explored['reviews'])
user_reviews_explored.reset_index(drop=True, inplace=True)
user_reviews_normalized.reset_index(drop=True, inplace=True)
user_reviews_final = pd.concat([user_reviews_explored.drop('reviews', axis=1), user_reviews_normalized], axis=1)

In [26]:
#creamos la funcion de analisis de sentimiento
def analizar_sentimiento(review):
    if pd.notnull(review):  # Verificar si la reseña no está ausente
        analysis = TextBlob(review)
        # Asignar valor según la polaridad de la reseña
        if analysis.sentiment.polarity < 0:
            return 0  # Malo
        elif analysis.sentiment.polarity == 0:
            return 1  # Neutral
        else:
            return 2  # Positivo
    else:
        return 1  # Valor predeterminado si la reseña está ausente

#creamos la columna "sentiment_analysis" en base a la columna "review" a la cual se le aplicó la funcion de arriba
user_reviews_final['sentiment_analysis'] = user_reviews_final['review'].apply(analizar_sentimiento)

In [27]:
#aquí renombramos las columnas para que sea vean más prolijas y esteticas
user_reviews_final.rename(columns={'sentiment_analysis': 'Sentiment Analysis'}, inplace=True)
user_reviews_final.rename(columns={'recommend': 'Recommend'}, inplace=True)
user_reviews_final.rename(columns={'helpful': 'Helpful'}, inplace=True)
user_reviews_final.rename(columns={'item_id': 'Item ID'}, inplace=True)
user_reviews_final.rename(columns={'last_edited': 'Last Edited'}, inplace=True)
user_reviews_final.rename(columns={'funny': 'Funny'}, inplace=True)
user_reviews_final.rename(columns={'user_url': 'User URL'}, inplace=True)
user_reviews_final.rename(columns={'user_id': 'User ID'}, inplace=True)

In [28]:
#creamos la columna "Year" que llenaremos con datos extraidos de la columna "posted"
user_reviews_final['Year'] = user_reviews_final['posted'].str.extract(r'(\d{4})')
#eliminamos la columna que ya no utilizamos
user_reviews_final.drop('posted' , axis = 1, inplace = True)

In [29]:
#eliminamos las siguientes columnas las cuales no utilizaremos
user_reviews_final.drop(columns="Funny", inplace=True)
user_reviews_final.drop(columns="Last Edited", inplace=True)

In [30]:
#eliminamos los datos nulos que hayan en la columna "Year"
user_reviews_final.dropna(subset="Year", inplace=True)

In [31]:
#Eliminamos los datos nulos de las columnas donde todas las filas sean nulas
user_reviews_final.dropna(how="all",inplace=True)

In [44]:
#vemos como van quedando los datos y la cantidad
user_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24249 entries, 2 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   User ID             24249 non-null  object
 1   Item ID             24249 non-null  object
 2   Recommend           24249 non-null  object
 3   Sentiment Analysis  24249 non-null  int64 
 4   Year                24249 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


In [35]:
#eliminamos las columnas que ya no utilizamos
user_reviews_final.drop('Helpful' , axis = 1, inplace = True)
user_reviews_final.drop('User URL' , axis = 1, inplace = True)
user_reviews_final.drop('review' , axis = 1, inplace = True)

In [38]:
#eliminamos datos duplicados que hayan en el dataframe
user_reviews_final.drop_duplicates(inplace=True)

In [39]:
#vemos como van quedando los datos y las columnas
user_reviews_final.head(2)

,User ID,Item ID,Recommend,Sentiment Analysis,Year
0,76561197970982479,1250,True,2,2011
1,76561197970982479,22200,True,2,2011


In [19]:
#convertimos el archivo a parquet
user_reviews_final.to_parquet("user_reviews_final.parquet")